In [1]:
ls

 Volume in drive C is OS
 Volume Serial Number is C803-CD91

 Directory of C:\Users\Miya\OneDrive\financial-programming\files\financial programming-HOMEWORK\2HFT

12/17/2016  09:00 PM    <DIR>          .
12/17/2016  09:00 PM    <DIR>          ..
12/17/2016  04:18 PM    <DIR>          .ipynb_checkpoints
12/09/2016  10:24 PM           216,053 2HFT.pdf
12/17/2016  08:53 PM        31,942,656 JNJ.db
12/09/2016  10:24 PM        38,242,798 TAQ_CAT_FEB_2010_trading_.csv
12/09/2016  10:24 PM        23,299,333 TAQ_CAT_QUOTE_0104_2010.csv
12/09/2016  10:24 PM        17,621,773 TAQ_JNJ_1004_1015_2010_trading_.csv
12/17/2016  08:53 PM        24,177,271 TAQ_JNJ_1004_1015_2010_trading_normal_hours.csv
12/17/2016  09:00 PM           169,518 Untitled.ipynb
               7 File(s)    135,669,402 bytes
               3 Dir(s)  81,777,041,408 bytes free


In [20]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pickle

In [1]:
with open('TAQ_CAT_QUOTE_0104_2010.csv') as f:
    content = f.read()
    data = content.split('\n')
    columns = list(filter(lambda x:x!='',data[0].split(' '))) 

In [2]:
columns

['DATE', 'Hour', 'Minute', 'Second', 'BID', 'OFR', 'BIDSIZ', 'OFRSIZ']

In [3]:
import sqlite3 as lite
con = lite.connect('quote.db')
with con:
    cur = con.cursor()
    cur.execute('drop table if exists quote')
    create_table_query = 'create table quote' + '(date,hour,minute,second,bid,ofr,bidsize,ofrsize)'
    cur.execute(create_table_query)

In [5]:
file_length = len(data)
for i in range(1, file_length):
    row = list(filter(lambda x:x!='',data[i].split(' ')))
    query = '''Insert into '''
    cur.execute('''Insert into quote values (?,?,?,?,?,?,?,?)''', row)

# Normal Transaction Window

In [12]:
with con:
    cur = con.cursor()
    cur.execute("""select * from quote""")
    all_data = cur.fetchall()

In [13]:
import pandas as pd
normal_df = pd.DataFrame(all_data,columns = columns)
normal_df.columns = ['date','hour','minute',
                     'second','bid','ofr','bidsize','ofrsize']
normal_df['bidsize'] = normal_df['bidsize'].astype(float)
normal_df['bid'] = normal_df['bid'].astype(float)
normal_df['ofr'] = normal_df['ofr'].astype(float)
normal_df['ofrsize'] = normal_df['ofrsize'].astype(float)
normal_df['time'] = normal_df['date']+normal_df['hour']+normal_df['minute']+normal_df['second']
normal_df['time'] = pd.to_datetime(normal_df['time'],format = '%Y%m%d%H%M%S')

In [15]:
normal_df['bid_offer_diff'] = normal_df['ofr'] - normal_df['bid']

In [16]:
normal_df.head()

,date,hour,minute,second,bid,ofr,bidsize,ofrsize,time,bid_offer_diff
0,20100104,4,07,00,56.97,57.89,5.0,5.0,2010-01-04 04:07:00,0.92
1,20100104,4,07,00,56.97,57.89,5.0,5.0,2010-01-04 04:07:00,0.92
2,20100104,4,07,01,56.98,57.89,5.0,5.0,2010-01-04 04:07:01,0.91
3,20100104,4,07,37,56.97,57.88,5.0,5.0,2010-01-04 04:07:37,0.91
4,20100104,4,07,48,56.97,0.00,5.0,0.0,2010-01-04 04:07:48,-56.97


# Partition trading time

In [18]:
intervals = ['5Min','10Min','30Min']

In [34]:
for interval in intervals:
    grouped = normal_df.groupby(pd.TimeGrouper(key = 'time',freq=interval)).aggregate(np.mean)
    grouped.to_csv('TAQ_CAT_QUOTE_0104_2010_'+interval+'trading_unit.csv')
    V_offer = grouped['ofrsize']
    p = V_offer/np.sum(V_offer)
    V_bid = grouped['bidsize']
    q = V_bid/np.sum(V_bid)
    p.replace({0:1})
    H_bid= -1*np.sum(q*np.log2(q))
    H_offer= -1*np.sum(p*np.log2(p))
    entropy = np.sum(p*np.log2(p/q))
    print('For %s trading unit, entropy value is %s'%(interval, entropy))

For 5Min trading unit, entropy value is 0.4319003226979326
For 10Min trading unit, entropy value is 0.41226993890260594
For 30Min trading unit, entropy value is 0.38046996298891983
